In [4]:
import numpy as np

In [5]:
def fitgarch1(y, lr= 1e-4, max_iter=50000):
    y = np.asarray(y, dtype=float)
    T = len(y)
    y = y - y.mean()
    def nll(theta):
        a,b,c = theta
        omega = np.exp(a)
        u = 1.0 /(1.0 + np.exp(-b))
        v = 1.0 /(1.0 + np.exp(-c))
        alpha = u * (1-v)
        beta = u * v
        sigma2 = np.empty(T)
        sigma2[0] = np.var(y)
        for t in range(1,T):
            sigma2[t] = omega + alpha * y[t-1]**2+beta* sigma2[t-1]
        return 0.5 * np.sum(np.log(sigma2)+(y**2)/sigma2)
    y_var = np.var(y)
    alpha0 = 0.1
    beta0  = 0.7
    omega0 = y_var * (1.0 - alpha0 - beta0)
    a0 = np.log(omega0)
    b0 = np.log(alpha0 /(1-alpha0))
    c0 = np.log(beta0 /(1-beta0))
    theta0 = np.array([a0,b0,c0], dtype = float)
    theta_hat, nll_min = gradient_descent(nll, theta0, lr= lr, max_iter = max_iter)
    a_hat, b_hat, c_hat = theta_hat
    omega_hat = np.exp(a_hat)
    alpha_hat = 1.0/(1.0+ np.exp(-b_hat))
    beta_hat = 1.0/(1.0+ np.exp(-c_hat))
    return {
        "omega": omega_hat,
        "alpha": alpha_hat,
        "beta": beta_hat,
        "nll": nll_min
    }

In [8]:
import yfinance as yf
data = yf.download("SPY")
spy_close = data["Close"]["SPY"]
prices = spy_close.to_numpy()
log_prices = np.log(prices)
returns = np.diff(log_prices)
y = returns -returns.mean()
res = fitgarch1(y)
print("Estimated omega:", res["omega"])
print("Estimated alpha:", res["alpha"])
print("Estimated beta:", res["beta"])

/var/folders/_b/28jfj19121qf32rdzd8twc000000gn/T/ipykernel_337/868467236.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("SPY")
[*********************100%***********************]  1 of 1 completed

Estimated omega: 6.952965832024371e-05
Estimated alpha: 0.08289674970156292
Estimated beta: 0.7698824945763842


In [7]:
def numerical_grad(f, x, eps=1e-5):
    x = np.asarray(x, dtype=float)
    g = np.zeros_like(x)
    for i in range(len(x)):
        x_step = x.copy()
        x_step[i] += eps
        f_plus = f(x_step)
        x_step[i] -= 2 * eps
        f_minus = f(x_step)
        g[i] = (f_plus - f_minus) / (2 * eps)
    return g

def gradient_descent(f, x0, lr=1e-3, max_iter=10000, tol=1e-6):
    x = np.asarray(x0, dtype=float)
    fx = f(x)
    for it in range(max_iter):
        g = numerical_grad(f, x)
        if np.linalg.norm(g) < tol:
            break
        x_new = x - lr * g
        fx_new = f(x_new)
        if fx_new > fx:
            lr *= 0.5
        else:
            x = x_new
            fx = fx_new
    return x, fx